In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
plt.style.use(['seaborn'])

from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.regression.mixed_linear_model import MixedLM, MixedLMResults
from sklearn.metrics import r2_score, mean_absolute_error
from scipy.stats import normaltest
from sklearn.preprocessing import quantile_transform

np.random.seed(42)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path='/DATA.xlsx'
soil_data_final=pd.read_excel(path)

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(25,10))
ax[0].hist(soil_data_final['Sac'], color='r')
ax[0].set_title('Histogram of Sac')
ax[1].hist(soil_data_final['Sac % Caña'],color='g')
ax[1].set_title('Histogram of Sac % Caña')
ax[2].hist(soil_data_final['Sac Campo'])
ax[2].set_title('Histogram of Sac Campo')

In [ ]:
def normal_test(target):
	stat, p = normaltest(soil_data_final[target])
	alpha =0.05
	if p > alpha:
		print(target + ' looks Gaussian')
	else:
		print(target + ' does not look Gaussian')
normal_test('Sac')
normal_test('Sac % Caña')
normal_test('Sac Campo')

There is no conclusion about the normality.

Using preprocessing techniques such as *Quantile transformation* we can normalize our targets.

In [ ]:
def normalization(target):
    y=soil_data_final[target]
    y_trans = quantile_transform(y.to_frame(), output_distribution="normal", copy=True)
    return y_trans

soil_data_final['sac_trans'] = normalization('Sac')
soil_data_final['sac_caña_trans']= normalization('Sac % Caña')
soil_data_final['sac_campo_trans'] = normalization('Sac Campo')

In [ ]:
# fig, ax = plt.subplots(1,3,figsize=(25,10))
# ax[0].hist(soil_data_final['sac_trans'], color='r')
# ax[0].set_title('Histogram of Sac after transformation')
# ax[1].hist(soil_data_final['sac_caña_trans'],color='g')
# ax[1].set_title('Histogram of Sac % Caña after transformation')
# ax[2].hist(soil_data_final['sac_campo_trans'])
# ax[2].set_title('Histogram of Sac Campo after transformation')

Info about target variables after the transformation

In [ ]:
# soil_data_final[['sac_trans', 'sac_campo_trans', 'sac_caña_trans']].describe()

Testing normality

In [ ]:
# normal_test('sac_trans')
# normal_test('sac_caña_trans')
# normal_test('sac_campo_trans')

Our targets seem more normal.

In [ ]:
[x.replace(' ','_') for x in soil_data_final.columns.tolist()]

In [ ]:
soil_data_final.drop(['Sac', 'Sac % Caña', 'Sac Campo'], axis=1, inplace=True)
soil_data_final.columns = [x.replace(' ','_') for x in soil_data_final.columns.tolist()]
data_train, data_test = train_test_split(soil_data_final, test_size=0.2)
data_train.reset_index(drop=True, inplace=True)
data_test.reset_index(drop=True, inplace=True)

In [ ]:
data_test.to_csv('data_test.csv')

In [ ]:
def formula_from_cols(columns, y, targets, groups):
    return y + ' ~ ' + ' + '.join([col for col in columns if col not in targets+groups])

groups = ['VAR']

formula_sac = formula_from_cols(soil_data_final.columns.tolist(),'sac_trans', ['sac_trans','sac_campo_trans','sac_caña_trans'],groups)
# formula_sac_campo = formula_from_cols(data_train,'sac_campo_trans', ['sac_trans','sac_campo_trans','sac_caña_trans'],groups)
# formula_sac_caña = formula_from_cols(data_train,'sac_caña_trans', ['sac_trans','sac_campo_trans','sac_caña_trans'])

In [ ]:
!pip install rpy2

In [ ]:
from rpy2.robjects import r

In [ ]:
r('print(')

In [ ]:
mixed_model_sac = smf.mixedlm(formula_sac, data_train, groups=data_train[]).fit()
# mixed_model_sac_campo = smf.glm(formula_sac_campo, data_train).fit()
# mixed_model_sac_caña = smf.glm(formula_sac_caña, data_train).fit()

In [ ]:
def plot_series(time, series,i,axis, format="-", start=0, end=None):
    #plt.figure(figsize=(20,10))
    axis.plot(time[start:end], series[start:end], format,label=i)
    # axis.set_xlabel("Unseen Samples")
    # axis.set_ylabel("Saccharose Field")
    axis.legend()

In [ ]:
mixed_model_sac.predict(data_test)

In [ ]:
data_test.reset_index(inplace=True)
fig, ax = plt.subplots(3,1,figsize=(25,15), sharex=True)
plot_series(data_test.index, data_test['sac_trans'], "True", ax[0])
plot_series(data_test.index, mixed_model_sac.predict(data_test.drop(['sac_campo_trans','sac_trans','sac_caña_trans'], axis=1)),'Predicted',ax[0])
ax[0].set_title('Sac true vs predicted')

In [ ]:
mixed_model_sac = smf.mixedlm(formula_sac, data_train, groups=data_train['VAR']).fit()

In [ ]:
mixed_model_sac.summary()

In [ ]:
formula_sac

In [ ]:
x = data_train.drop(['sac_trans'], axis=1)
y = data_train['sac_trans']
x = sm.add_constant(x)

In [ ]:
one_hot_vars = ['tmprda', 'TIPO COS','Con Sin Mad', 'nm_cndcion', 'PRODUCTO', 'VAR']
data_proccessed = pd.get_dummies(soil_data_final, columns = one_hot_vars)
data_proccessed = data_proccessed.drop(['TIPO COS_manual','Con Sin Mad_Sin Mad'], axis=1)   #variables who are binary are deleted and keep only one.
possible_targets = ['Sac', 'Sac Campo', 'Sac % Caña']
y = data_proccessed[possible_targets]
X = data_proccessed.drop(possible_targets, axis=1)
X = sm.add_constant(X) #add 1 to match bias later
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) #split data into training and testing(20%)
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [ ]:
sns.clustermap(soil_data_final.corr())

In [ ]:
mixed = MixedLM(y_train['Sac'], X_train, groups=X_train['Pza']).fit()

In [ ]:
pred_sac = mixed.predict(X_test)
fig, ax = plt.subplots(3,1,figsize=(25,15), sharex=True)
plot_series(y_test.index, y_test['sac_trans'], "True", ax[0])
plot_series(y_test.index, pred_sac_,'Predicted',ax[0])
ax[0].set_title('Sac true vs predicted')

In [ ]:
y_train.describe()